# PyTorch 및 SMDataParallel을 사용한 분산 데이터 병렬 MaskRCNN 훈련

SMDataParallel은 Amazon SageMaker의 새로운 기능으로 딥러닝 모델을 더 빠르고 저렴하게 훈련할 수 있습니다. SMDataParallel은 PyTorch, TensorFlow 및 MXNet을 위한 분산 데이터 병렬 훈련 프레임워크입니다.

이 노트북 예제는 [Amazon SageMaker](https://aws.amazon.com/sagemaker/)에서 PyTorch(버전 1.6.0)와 함께 SMDataParallel을 사용하여 [Amazon FSx for Lustre file-system](https://aws.amazon.com/fsx/lustre/) 파일 시스템을 데이터 소스로 사용하는 MaskRCNN 모델 훈련 방법을 보여줍니다. 

본 예제의 개요는 다음과 같습니다.

1. [Amazon S3](https://aws.amazon.com/s3/)에서 COCO 2017 데이터셋을 준비합니다.
2. Amazon FSx Luster 파일 시스템을 생성하고 S3에서 파일 시스템으로 데이터를 가져옵니다.
3. Docker 훈련 이미지를 빌드하고 [Amazon ECR](https://aws.amazon.com/ecr/)에 푸시합니다.
4. SageMaker에 대한 데이터 입력 채널을 구성합니다.
5. 하이퍼 파라메터를 세팅합니다.
6. 훈련 지표를 정의합니다.
7. 훈련 작업을 정의하고 분산 전략을 SMDataParallel로 설정하고 훈련을 시작합니다.

**참고:** 대규모 훈련 데이터셋의 경우 SageMaker 훈련 작업을 위한 입력 파일 시스템으로 (Amazon FSx)[https://aws.amazon.com/fsx/] 를 사용하는 것이 좋습니다. SageMaker에 대한 FSx 파일 입력은 SageMaker 훈련 작업을 시작할 때마다 훈련 데이터 다운로드를 방지하고 (SageMaker 훈련 작업에 대한 S3 입력으로 수행됨) 우수한 데이터 읽기 처리량(throughput)을 제공하므로 SageMaker에서 훈련 시작 시간을 크게 단축합니다.

**참고:** 이 예제는 SageMaker Python SDK v2.X가 필요합니다.

## Amazon SageMaker 초기화

노트북 인스턴스를 초기화합니다. aws 리전, sagemaker 실행 역할을 가져옵니다.

IAM 역할 arn은 데이터에 대한 훈련 및 호스팅 액세스 권한을 부여하는 데 사용됩니다. 이를 생성하는 방법은 [Amazon SageMaker 역할](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html)을 참조하세요. 노트북 인스턴스, 훈련 및 호스팅에 둘 이상의 역할이 필요한 경우 `sagemaker.get_execution_role()`을 적절한 전체 IAM 역할 arn 문자열로 변경해 주세요. 위에서 설명한 대로 FSx를 사용할 것이므로, 이 IAM 역할에 `FSx Access` 권한을 연결해야 합니다.

In [ ]:
%%time
! python3 -m pip install --upgrade sagemaker
import sagemaker
from sagemaker import get_execution_role
from sagemaker.estimator import Estimator
import boto3

sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()

role = get_execution_role() # provide a pre-existing role ARN as an alternative to creating a new role
print(f'SageMaker Execution Role:{role}')

client = boto3.client('sts')
account = client.get_caller_identity()['Account']
print(f'AWS account:{account}')

session = boto3.session.Session()
region = session.region_name
print(f'AWS region:{region}')

## SageMaker 훈련 이미지 준비

1. SageMaker는 기본적으로 최신 [Amazon Deep Learning Container Images (DLC)](https://github.com/aws/deep-learning-containers/blob/master/available_images.md) PyTorch 훈련 이미지를 사용합니다. 이 단계에서는 이를 기본 이미지로 사용하고 MaskRCNN 모델 훈련에 필요한 추가 종속성 패키지들을 설치합니다.
2. Github 저장소 https://github.com/HerringForks/DeepLearningExamples.git 에서 PyTorch-SMDataParallel BERT 훈련 스크립트를 사용할 수 있도록 만들었습니다. 이 저장소는 모델 훈련을 실행하기 위해 훈련 이미지에서 복제됩니다.

### Docker 이미지 빌드 및 ECR로 푸시

아래 명령을 실행하여 도커 이미지를 빌드하고 ECR에 푸시합니다.

In [ ]:
image = "<ADD NAME OF REPO>"  # Example: mask-rcnn-smdataparallel-sagemaker
tag = "<ADD TAG FOR IMAGE>"   # Example: pt1.6 

In [ ]:
!pygmentize ./Dockerfile

In [ ]:
!pygmentize ./build_and_push.sh

In [ ]:
%%time
! chmod +x build_and_push.sh; bash build_and_push.sh {region} {image} {tag}

## SageMaker용 FSx 입력 데이터 준비

1. S3에서 훈련 데이터셋을 다운로드하고 준비합니다.
2. 여기에 나열된 단계에 따라 훈련 데이터(https://docs.aws.amazon.com/fsx/latest/LustreGuide/create-fs-linked-data-repo.html)가 있는 S3 버켓과 연결된 FSx를 생성합니다. S3 액세스를 허용하는 엔드포인트를 VPC에 추가해야 합니다.
3. 여기에 나열된 단계에 따라 FSx(https://aws.amazon.com/blogs/machine-learning/speed-up-training-on-amazon-sagemaker-using-amazon-efs-or-amazon-fsx-for-lustre-file-systems/)를 사용하도록 SageMaker 훈련 작업을 구성합니다. 


### 중요 사항

1. SageMaker 노트북 인스턴스를 시작할 때 FSx에서 사용하는 것과 동일한 `서브넷(subnet)` 과`vpc` 및 `보안 그룹(security group)`을 사용해야 합니다. SageMaker 훈련 작업에서 동일한 구성이 사용됩니다.
2. '보안 그룹'에서 적절한 인바운드/출력 규칙을 설정했는지 확인합니다. 특히 SageMaker가 훈련 작업에서 FSx 파일 시스템에 액세스하려면 이러한 포트를 열어야합니다. https://docs.aws.amazon.com/fsx/latest/LustreGuide/limit-access-security-groups.html
3. 이 SageMaker 훈련 작업을 시작하는 데 사용된 `SageMaker IAM 역할`이 `AmazonFSx`에 액세스할 수 있는지 확인합니다.

## SageMaker PyTorch Estimator function options

다음 코드 블록에서 다른 인스턴스 유형, 인스턴스 수 및 분산 전략을 사용하도록 estimator 함수를 업데이트할 수 있습니다. 이전 코드 셀에서 검토한 훈련 스크립트도 estimator 함수로 전달합니다.

**인스턴스 유형**

SMDataParallel은 아래 인스턴스 유형들만 SageMaker 상에서의 모델 훈련을 지원합니다.
1. ml.p3.16xlarge
1. ml.p3dn.24xlarge [권장]
1. ml.p4d.24xlarge [권장]

**인스턴스 수**

최상의 성능과 SMDataParallel을 최대한 활용하려면 2개 이상의 인스턴스를 사용해야 하지만, 이 예제를 테스트하는 데 1개를 사용할 수도 있습니다.

**배포 전략**

DDP 모드를 사용하려면 `distribution` 전략을 업데이트하고 `smdistributed dataparallel`을 사용하도록 설정해야 합니다.


In [ ]:
import os
from sagemaker.pytorch import PyTorch

In [ ]:
instance_type = "ml.p3dn.24xlarge" # Other supported instance type: ml.p3.16xlarge
instance_count = 2 # You can use 2, 4, 8 etc.
docker_image = f"{account}.dkr.ecr.{region}.amazonaws.com/{image}:{tag}" # YOUR_ECR_IMAGE_BUILT_WITH_ABOVE_DOCKER_FILE
region = '<REGION>' # Example: us-west-2
username = 'AWS'
subnets=['<SUBNET_ID>'] # Should be same as Subnet used for FSx. Example: subnet-0f9XXXX
security_group_ids=['<SECURITY_GROUP_ID>'] # Should be same as Security group used for FSx. sg-03ZZZZZZ
job_name = 'pytorch-smdataparallel-mrcnn-fsx' # This job name is used as prefix to the sagemaker training job. Makes it easy for your look for your training job in SageMaker Training job console.
file_system_id= '<FSX_ID>' # FSx file system ID with your training dataset. Example: 'fs-0bYYYYYY'
config_file = 'e2e_mask_rcnn_R_50_FPN_1x_64GPU_4bs.yaml'

In [ ]:
hyperparameters = {
 "config-file": config_file,
 "skip-test": "",
 "seed": 987,
 "dtype": "float16",
 }

In [ ]:
estimator = PyTorch(entry_point='train_pytorch_smdataparallel_maskrcnn.py',
                        role=role,
                        image_uri=docker_image,
                        source_dir='.',
                        instance_count=instance_count,
                        instance_type=instance_type,
                        framework_version='1.6.0',
                        py_version='py3',
                        sagemaker_session=sagemaker_session,
                        hyperparameters=hyperparameters, 
                        subnets=subnets, 
                        security_group_ids=security_group_ids,
                        debugger_hook_config=False,
                        # Training using SMDataParallel Distributed Training Framework
                        distribution={'smdistributed':{
                                            'dataparallel':{
                                                    'enabled': True
                                                 }
                                          }
                                      }
                   )

In [ ]:
# Configure FSx Input for your SageMaker Training job

from sagemaker.inputs import FileSystemInput
file_system_directory_path= 'YOUR_MOUNT_PATH_FOR_TRAINING_DATA' # NOTE: '/fsx/' will be the root mount path. Example: '/fsx/mask_rcnn/PyTorch'
file_system_access_mode='ro'
file_system_type='FSxLustre'
train_fs = FileSystemInput(file_system_id=file_system_id,
                                    file_system_type=file_system_type,
                                    directory_path=file_system_directory_path,
                                    file_system_access_mode=file_system_access_mode)
data_channels = {'train': train_fs}

In [ ]:
# Submit SageMaker training job
estimator.fit(inputs=data_channels, job_name=job_name)